In [11]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import numpy as np
import time

# ДЗ 2 Selenium

Сайт = https://1000.menu/

Код ищет рецепты для блюд в списке search. Затем он заходит в каждый отдельный рецепт и там находит список ингридиентов и основные нутриенты. 

In [1]:
search = ['тирамису', 'маффин', 'торт', 'пирожное', "пирог", "варенье"]

Код находит для каждого блюда калорийность, время приготовления, ссылку, имя, и описания рецептов в результате. 

In [ ]:
driver = webdriver.Chrome()
queries = []
calories = []
times = []
links = []
names = []
descriptions = []
for query in search:
    url = f'https://1000.menu/cooking/search?ms=1&str={query}'
    try:
        driver.get(url)
    except Exception as e:
        print(f"An error occurred: {e}")

    for _ in range(5): 
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(1)
        
    recipes = driver.find_elements(By.CLASS_NAME, "cn-item")
    
    for recipe in recipes:  
        try:
            left = recipe.find_element(By.CLASS_NAME, "level-left")
            calories.append(left.text) if left else calories.append(np.nan)
            right = recipe.find_element(By.CLASS_NAME, "level-right")
            times.append(right.text) if right else times.append(np.nan)
            heading = recipe.find_element(By.CLASS_NAME, "h5")
            link = heading.get_attribute('href')
            links.append(link) if link else links.append(np.nan)
            name = heading.text
            names.append(name)
            preview = recipe.find_element(By.CLASS_NAME, "preview-text")
            descriptions.append(preview.text)
            queries.append(query)
        except: 
            all_recipes = pd.DataFrame({
                'calories':calories,
                'time':times,
                'link':links,
                'name':names,
                'description':descriptions,
                'query':queries
            })
            pass
            

driver.quit()

In [ ]:
all_recipes

Убираем дубликаты

In [111]:
all_recipes.drop_duplicates(subset=['link'], inplace=True, ignore_index=True)

In [64]:
mega_ingredients = []
mega_macros = []

Код заходит на каждую ссылку и берет список ингредиентов и список белков, жиров, углеводов

In [69]:
driver = webdriver.Chrome()
all_links = []
for link in all_recipes.link:
    ingredients_list = []
    macros = []
    url = link
    try:
        driver.get(url)
    except Exception as e:
        print(f"An error occurred: {e}")

    for _ in range(2): 
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)
    try:
        ingredients = driver.find_elements(By.CLASS_NAME, "ingredient.list-item")
        for ingredient in ingredients:
            meta = ingredient.find_element(By.CSS_SELECTOR, '[itemprop="recipeIngredient"]')
            ingredients_list.append(meta.get_attribute('content'))
    except: 
        ingredients_list.append(np.nan)
    try:
        calories = driver.find_elements(By.CLASS_NAME, "percent")
        for macro in calories:
            macros.append(macro.text)
    except:
        macros.append(np.nan)
    mega_ingredients.append(ingredients_list)
    mega_macros.append(macros)
    all_links.append(link)

driver.quit()


In [ ]:
ingredients_and_macros = pd.DataFrame({
    'link':all_links,
    'macros':mega_macros,
    'ingredients':mega_ingredients
})

Получается одна большая таблица

In [166]:
all_recipes = pd.merge(all_recipes, ingredients_and_macros, on='link', how='left')

In [168]:
all_recipes.to_csv('all_recipes.csv', encoding="utf-8")